In [1]:
import pandas as pd
import requests
import json
import time

In [2]:
bikes = pd.read_csv("CDMX_bikes.csv")
bikes.head()

,uid,latitude,longitude,empty_slots,free_bikes,ll
0,109,19.437400,-99.137571,15,4,"19.4374,-99.137571"
1,272,19.436531,-99.131497,17,6,"19.436531,-99.131497"
2,108,19.436741,-99.134811,23,12,"19.436741,-99.134811"
3,15,19.433766,-99.130918,16,7,"19.433766,-99.130918"
4,256,19.430722,-99.133698,14,5,"19.430722,-99.133698"


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

API documentation: "categories"
museum (all) = 10027
bar (all) = 13003 
restaurant (all)= 13065

I'm also going to limit the output using param "fields"

In [3]:
full_foursquare_data = []
for bike_loc in range(len(bikes)):

    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "ll" : bikes["ll"].iloc[bike_loc],
        "categories" : "13003,10027,13065",
        "radius" : "50",
        "sort":"DISTANCE",
        "limit" : "50",
        "fields" : "name,geocodes,categories,price,rating,distance"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3qnvtw345lsOhc+snv7hneg8d02DFLOe2i9PR0u82sp0="
    }

    response = requests.request("GET", url, params=params, headers=headers)
    jsoned = response.json()

    full_foursquare_data.append(jsoned)
response

<Response [200]>

Manually breaking apart the list looks like this:

```{'results': [
    {
        'categories': 
            [
                {'id': 10028, 
                'name': 'Art Museum', 
                'icon': 
                    {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_', 
                    'suffix': '.png'  }
                }
            ], 
        'geocodes': 
            {
            '   main': 
                    {'latitude': 19.433658, 
                   'longitude': -99.131113  }, 
                'roof': 
                    {'latitude': 19.433658, 
                    'longitude': -99.131113  }
            },
        'name': 'Museo de la Secretaría de Hacienda y Crédito Público'
    }, 
```
geopandas

In [13]:
POIs = []
counter = 0

for each_bike_loc in full_foursquare_data:
    foursq_results = each_bike_loc["results"]
    bike_ll = bikes["ll"][counter]

    counter += 1
    
    for result in foursq_results:
        each_POI = {}
        each_POI["POI_name"] = result["name"] 
        each_POI["latitude"] = result["geocodes"]["main"]["latitude"] 
        each_POI ["longitude"] = result["geocodes"]["main"]["longitude"] 
        each_POI ["category_name"] = result["categories"][0]["name"] 
        each_POI ["category_id"] = result["categories"][0]["id"] 

        if "price" in result:
            each_POI ["price"] = result ["price"]
        else:
            each_POI ["price"] = "NaN"
            
        if "rating" in result:
            each_POI ["rating"] = result ["rating"]
        else:
            each_POI ["rating"] = "NaN"
        each_POI ["distance"] = result["distance"]
        each_POI["bike_ll"] = bike_ll
        
        POIs.append(each_POI)

Put your parsed results into a DataFrame

In [14]:
df = pd.DataFrame(POIs)
df.head()

,POI_name,latitude,longitude,category_name,category_id,price,rating,distance,bike_ll
0,Cueva Colomba,19.437340,-99.137459,Bar,13003,2,6.1,13,"19.4374,-99.137571"
1,Tintico,19.437210,-99.137425,Coffee Shop,13035,1,7.6,16,"19.4374,-99.137571"
2,La Cueva Colomba,19.437141,-99.137434,Pub,13018,1,6.1,18,"19.4374,-99.137571"
3,Tintico Cafe & Galeria,19.437250,-99.137483,Café,13034,NaN,NaN,19,"19.4374,-99.137571"
4,Cantina Río de la Plata,19.437195,-99.137617,Beer Bar,13006,1,6.6,20,"19.4374,-99.137571"


In [15]:
print(df.shape)
df.to_csv("foursquare_df.csv")

(1689, 9)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [10]:
full_yelp_data = []
for bike_loc in range(len(bikes)):
    
    api_key = "q5Ks_s0oQ8NWcDRMGsa3Ca7B854LZ9NkRrYkcRwOKRAe-Si3YUD6ReCs10sSC3YXnWDCcdypoj5Scl6jjO75W2DwMhAl0lIj6KeKwSSd51w2k0TbXhhqhIFByubVY3Yx"
    
    headers = {
        "Authorization": f"Bearer {api_key}"
        }

    parameters = {
        "term": "restaurant,bar",
        "radius": "50",
        "latitude":  bikes["latitude"].iloc[bike_loc],
        "longitude": bikes["longitude"].iloc[bike_loc]
        }

    response = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=parameters)
    jsoned = response.json()

    full_yelp_data.append(jsoned)
response

<Response [200]>

```{'businesses': 
[
   {'id': 'gzypZfzJi4R6vEnc16m5BA',
   'alias': 'la-imperial-ciudad-de-mexico-2',
   'name': 'La Imperial',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/K3KrUAQXglyRMo-NxPT0jQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/la-imperial-ciudad-de-mexico-2?adjust_creative=29zC3K-Gh-e9CGAFP_YpFA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=29zC3K-Gh-e9CGAFP_YpFA',
   'review_count': 8,
   'categories': [{'alias': 'mexican', 'title': 'Mexican'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 19.44195, 'longitude': -99.20296},
   'transactions': [],
   'price': '$$$',
   'location': {'address1': 'Calle Lago Zurich 245',
    'address2': '',
    'address3': '',
    'city': 'Ciudad de Mexico',
    'zip_code': '11529',
    'country': 'MX',
    'state': 'MEX',
    'display_address': ['Calle Lago Zurich 245',
     '11529 Ciudad de Mexico, México',
     'Mexico']},
   'phone': '+525549760336',
   'display_phone': '+52 55 4976 0336',
   'distance': 359.0865727554903},
```

In [16]:
POIs = []
counter = 0

for each_bike_loc in full_yelp_data:
    yelp_results = each_bike_loc["businesses"]
    bike_ll = bikes["ll"][counter]

    counter += 1
    
    for result in yelp_results:
        each_POI = {}
        each_POI["POI_name"] = result["name"] 
        each_POI["latitude"] = result["coordinates"]["latitude"] 
        each_POI ["longitude"] = result["coordinates"]["longitude"] 
        each_POI ["category_name"] = result["categories"][0]["title"] 

        if "price" in result:
            each_POI ["price"] = result ["price"]
        else:
            each_POI ["price"] = "NaN"
        
        if "rating" in result:
            each_POI ["rating"] = result["rating"]
        else:
            each_POI ["rating"] = "NaN"
        each_POI ["distance"] = round(result["distance"], 0)

        each_POI["bike_ll"] = bike_ll
        
        POIs.append(each_POI)

Put your parsed results into a DataFrame

In [17]:
df2 = pd.DataFrame(POIs)
print(df2.shape)
df2.to_csv("yelp_df.csv")
df2.head()

(63, 8)


,POI_name,latitude,longitude,category_name,price,rating,distance,bike_ll
0,Casa de la Luz,19.429510,-99.132980,Hotels,NaN,4.5,49.0,"19.429189,-99.132759"
1,Marisqueria Victoria,19.427342,-99.136988,Juice Bars & Smoothies,$,5.0,34.0,"19.427059,-99.137116"
2,Sartoria,19.421983,-99.160581,Italian,NaN,3.5,20.0,"19.422148,-99.160645"
3,Buna,19.421939,-99.160531,Cafeteria,$,4.5,26.0,"19.422148,-99.160645"
4,Café Toscano,19.421867,-99.160464,Cafeteria,$$,3.5,37.0,"19.422148,-99.160645"


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Foursquare gave more results (1689 v 63), had fewer categories and more finely tuned ratings (out of 10 rather than 5))

Get the top 10 restaurants according to their rating

In [18]:
df["rating"] = df["rating"].astype(float)
df.sort_values(by="rating", ascending=False).head(10)


,POI_name,latitude,longitude,category_name,category_id,price,rating,distance,bike_ll
843,Museo Nacional de Arte (Museo Nacional de Arte...,19.436255,-99.139577,Art Museum,10028,1,9.5,60,"19.436522,-99.139811"
1632,El Cardenal,19.434926,-99.146263,Mexican Restaurant,13303,3,9.4,22,"19.4351365,-99.1460756"
845,El Cardenal,19.433723,-99.135349,Mexican Restaurant,13303,3,9.2,12,"19.433757,-99.135331"
1431,Nevería Roxy,19.408810,-99.176618,Ice Cream Parlor,13046,2,9.2,22,"19.408676,-99.176759"
229,Tacos Orinoco,19.417872,-99.163014,Taco Restaurant,13306,1,9.0,34,"19.417715,-99.162729"
119,BLOM Café,19.425221,-99.159520,Coffee Shop,13035,1,8.9,20,"19.425437,-99.159458"
438,Catamundi,19.431082,-99.195686,Bistro,13027,3,8.9,47,"19.431417,-99.195774"
1604,Quintonil,19.430935,-99.191651,Mexican Restaurant,13303,4,8.9,47,"19.431242,-99.191613"
858,Mercaderes,19.433958,-99.134938,Restaurant,13065,3,8.9,58,"19.433757,-99.135331"
954,Museo Nacional de San Carlos,19.438225,-99.151914,Museum,10027,1,8.9,43,"19.438228,-99.152269"


In [19]:
df2.sort_values(by="rating", ascending=False).head(10)

,POI_name,latitude,longitude,category_name,price,rating,distance,bike_ll
23,Gael,19.406860,-99.172280,Hostels,NaN,5.0,34.0,"19.407233,-99.172131"
54,Tasty Bites Food Tours,19.419924,-99.166422,Food Tours,NaN,5.0,47.0,"19.41955,-99.166214"
41,Domingo Santo,19.437132,-99.134074,Cafeteria,NaN,5.0,26.0,"19.436931,-99.133952"
40,Gael,19.406860,-99.172280,Hostels,NaN,5.0,44.0,"19.40655,-99.172275"
10,Casa Jacaranda,19.418361,-99.159298,Cooking Classes,$$,5.0,28.0,"19.418304,-99.15956"
1,Marisqueria Victoria,19.427342,-99.136988,Juice Bars & Smoothies,$,5.0,34.0,"19.427059,-99.137116"
28,Hiyoko,19.429565,-99.169623,Japanese,$$$,5.0,32.0,"19.429804,-99.169451"
0,Casa de la Luz,19.429510,-99.132980,Hotels,NaN,4.5,49.0,"19.429189,-99.132759"
58,Quintonil,19.431174,-99.191560,Mexican,$$$$,4.5,9.0,"19.431242,-99.191613"
53,Temporal,19.410302,-99.173100,Bars,$$,4.5,36.0,"19.410468,-99.172802"
